# Líneas base de las sedes

In [1]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

In [2]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos' / 'Eficiencia_Energetica' / 'Homecenter' / 'Informe_01'
sys.path.append(str(project_path))

In [3]:
# import all your modules here
import json
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import config as cfg

from library_ubidots_v2 import Ubidots as ubi
from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro


In [4]:
df = pd.read_pickle(project_path / "dataframe.pkl")
df_tag_aa_ilu = pd.read_excel(project_path / "ubidots_device_map.xlsx", sheet_name='AA-ILU')

# for some reason 'value' dtypes aren't consistent...
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# merge both tables
df = (
    pd.merge(
        df.reset_index(),
        df_tag_aa_ilu,
        how='inner'
    )
    .set_index('datetime')
) 

In [5]:
dates_to_remove = {
    
    'cali-norte---torre-de-enfriamiento':[
        ['2022-01-01','2022-05-25']
    ],
    'cali-sur---aires-acondicionados':[
        ['2022-01-01','2022-05-25']
    ],
    'cedritos---iluminacion-patio-constructor':[
        ['2022-01-01','2022-05-20']
    ],
    
    'cali-norte---iluminacion-parqueaderos-2':[
        ['2022-01-01','2022-04-30']
    ],
    'cali-norte---iluminacion-principal-2':[
        ['2022-01-01','2022-05-25']
    ],
    'iluminacion---primer-piso':[ # cedritos
        ['2022-01-01','2022-12-30']
    ],
    'cedritos-iluminacion-principal':[
        ['2022-05-15','2022-12-30']
    ],
    'bello---iluminacion-patio-constructor':[
        ['2022-01-01','2022-05-25']
    ],
    'tintal---iluminacion-parqueaderos':[
        ['2022-01-01','2022-06-20']
    ],
    'baq---aire-acondicionado-piso-ventas':[
        ['2022-01-01','2022-05-20']
    ],
}

energy_var_rename = {
    'ener':'energia-activa-acumulada',
    'energia-activa':'energia-activa',
    'energia-activa-acumulada':'energia-activa-acumulada',
    'energia-activa-calculada':'energia-activa'
}

In [6]:
# get (unified) cummulative energy from consumption
df_cons = df.query('variable == "energia-activa"').copy()
df_ea = cln.recover_energy_from_consumption(df_cons, new_varname='energia-activa-acumulada')

# get power from energy
df_pa_synth = cln.differentiate_single_variable(df_ea, 'potencia-activa-sintetica', remove_gap_data=True)

# clean power data
df_pa_synth = cln.remove_outliers_by_zscore(df_pa_synth, zscore=5)
df_pa_synth = cln.subset_discard_date_intervals(df_pa_synth, dates_to_remove)
df_pa_synth = pro.datetime_attributes(df_pa_synth)
df_pa_synth = df_pa_synth.query("value >= 0")

In [7]:
df_pa_synth

,value,variable,device,device_name,type,office_label,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,,,
2022-04-01 02:00:00-05:00,0.18,potencia-activa-sintetica,san-fernando---iluminacion-patio-constructor,San Fernando - Iluminación Patio Constructor,ILU,hc-san-fernando,2,1,viernes,4.083333,13,4,2022
2022-04-01 05:00:00-05:00,4.82,potencia-activa-sintetica,san-fernando---iluminacion-patio-constructor,San Fernando - Iluminación Patio Constructor,ILU,hc-san-fernando,5,1,viernes,4.208333,13,4,2022
2022-04-01 06:00:00-05:00,17.30,potencia-activa-sintetica,san-fernando---iluminacion-patio-constructor,San Fernando - Iluminación Patio Constructor,ILU,hc-san-fernando,6,1,viernes,4.250000,13,4,2022
2022-04-01 10:00:00-05:00,25.32,potencia-activa-sintetica,san-fernando---iluminacion-patio-constructor,San Fernando - Iluminación Patio Constructor,ILU,hc-san-fernando,10,1,viernes,4.416667,13,4,2022
2022-04-01 11:00:00-05:00,25.31,potencia-activa-sintetica,san-fernando---iluminacion-patio-constructor,San Fernando - Iluminación Patio Constructor,ILU,hc-san-fernando,11,1,viernes,4.458333,13,4,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-27 07:00:00-05:00,0.68,potencia-activa-sintetica,san-fernando---iluminacion-parqueaderos,San Fernando - Iluminación Parqueaderos,ILU,hc-san-fernando,7,27,jueves,3.291667,43,10,2022
2022-10-27 13:00:00-05:00,4.37,potencia-activa-sintetica,san-fernando---iluminacion-parqueaderos,San Fernando - Iluminación Parqueaderos,ILU,hc-san-fernando,13,27,jueves,3.541667,43,10,2022
2022-10-27 14:00:00-05:00,2.45,potencia-activa-sintetica,san-fernando---iluminacion-parqueaderos,San Fernando - Iluminación Parqueaderos,ILU,hc-san-fernando,14,27,jueves,3.583333,43,10,2022


## Resultados:

<!-- Plotly curva horaria por circuito en la sede:
- Para AA
- Para Iluminación
- Para el total

Curva de KPI (kWh/m^2/año) por sede

Series de tiempo de consumos mensuales (bar plot)

Figura de consumo energético por tipo de día de la semana



Bell y San Juan dejan la iluminación prendida -->


In [8]:
def office_cell_function(df_office, office_name):

    df_day_by_hour = (
        df_office
        .reset_index()
        .groupby(['office_label','type','device_name','hour'])['value']
        .agg(['median','mean','std','min',pro.q_low,pro.q_high,'max','count'])
        .reset_index()
    )

    df_week_by_day = (
        df_office
        .reset_index()
        .groupby(['office_label','type','device_name','cont_dow'])['value']
        .agg(['median','mean','std','min',pro.q_low,pro.q_high,'max','count'])
        .reset_index()
    )

    grp.plot_typical_day_by_hour(
        df_day_by_hour.query("type == 'AA'"), 
        subset='device_name', 
        title=f"Día típico para los equipos de climatización en la sede de {office_name}", 
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

    grp.plot_typical_day_by_hour(
        df_day_by_hour.query("type == 'ILU'"), 
        subset='device_name', 
        title=f"Día típico para la iluminación en la sede de {office_name}", 
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )
    
    grp.plot_typical_week_by_day(
        df_week_by_day.query("type == 'AA'"),
        subset='device_name',  
        title=f"Semana típica para los equipos de climatización en la sede de {office_name}",
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

    grp.plot_typical_week_by_day(
        df_week_by_day.query("type == 'ILU'"),
        subset='device_name',  
        title=f"Semana típica para la iluminación en la sede de {office_name}",
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

In [12]:
df_cons = pro.datetime_attributes(df_cons)

In [13]:
df_cons

,value,variable,device,device_name,type,office_label,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,,,
2022-10-27 14:00:00-05:00,50.69,energia-activa,cali-norte---iluminacion-patio-constructor,Cali Norte - Iluminación Patio Constructor,ILU,hc---cali-norte,14,27,jueves,3.583333,43,10,2022
2022-10-27 12:00:00-05:00,84.50,energia-activa,cali-norte---iluminacion-patio-constructor,Cali Norte - Iluminación Patio Constructor,ILU,hc---cali-norte,12,27,jueves,3.500000,43,10,2022
2022-10-27 10:00:00-05:00,42.25,energia-activa,cali-norte---iluminacion-patio-constructor,Cali Norte - Iluminación Patio Constructor,ILU,hc---cali-norte,10,27,jueves,3.416667,43,10,2022
2022-10-27 09:00:00-05:00,8.45,energia-activa,cali-norte---iluminacion-patio-constructor,Cali Norte - Iluminación Patio Constructor,ILU,hc---cali-norte,9,27,jueves,3.375000,43,10,2022
2022-10-27 08:00:00-05:00,51.11,energia-activa,cali-norte---iluminacion-patio-constructor,Cali Norte - Iluminación Patio Constructor,ILU,hc---cali-norte,8,27,jueves,3.333333,43,10,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04 21:00:00-05:00,96.00,energia-activa,san-juan---iluminacion-patio-constructor,San Juan - Iluminación Patio Constructor,ILU,hc-san-juan,21,4,lunes,0.875000,14,4,2022
2022-04-04 20:00:00-05:00,52.00,energia-activa,san-juan---iluminacion-patio-constructor,San Juan - Iluminación Patio Constructor,ILU,hc-san-juan,20,4,lunes,0.833333,14,4,2022
2022-04-04 19:00:00-05:00,183.00,energia-activa,san-juan---iluminacion-patio-constructor,San Juan - Iluminación Patio Constructor,ILU,hc-san-juan,19,4,lunes,0.791667,14,4,2022


In [ ]:

(
    df_ea
    .groupby(by=['office_label','','',''])
)

### Barranquilla

In [ ]:
office_name = 'Barranquilla'
office_label = 'hc-baq'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Bello

In [ ]:
office_name = 'Bello'
office_label = 'hc-bello'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Bucaramanga

In [ ]:
office_name = 'Bucaramanga'
office_label = 'hc---bucaramanga'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Cali Norte

In [ ]:
office_name = 'Cali Norte'
office_label = 'hc---cali-norte'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Cali Sur

In [ ]:
office_name = 'Cali Sur'
office_label = 'hc-cali-sur'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Calle 80

In [ ]:
office_name = 'Calle 80'
office_label = 'calle-80'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Cedritos

In [ ]:
office_name = 'Cedritos'
office_label = 'cedritos'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Funza

In [ ]:
office_name = 'Funza'
office_label = 'hc-funza'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### La Popa

In [ ]:
office_name = 'La Popa'
office_label = 'hc-la-popa'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Palmira

In [ ]:
office_name = 'Palmira'
office_label = 'hc---palmira'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### San Fernando

In [ ]:
office_name = 'San Fernando'
office_label = 'hc-san-fernando'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### San Juan

In [ ]:
office_name = 'San Juan'
office_label = 'hc-san-juan'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Tintal

In [ ]:
office_name = 'Tintal'
office_label = 'hc-tintal'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)